In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
import tensorflow 
from tensorflow import keras
from keras import Sequential 
from keras.layers import Dense

In [11]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train, batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 [==============================] - 1s 14ms/step - loss: 0.6599 - accuracy: 0.6287 - val_loss: 0.6711 - val_accuracy: 0.5649
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6121 - accuracy: 0.6873 - val_loss: 0.6258 - val_accuracy: 0.6623
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5796 - accuracy: 0.7117 - val_loss: 0.5951 - val_accuracy: 0.7273
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5551 - accuracy: 0.7248 - val_loss: 0.5681 - val_accuracy: 0.7468
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5350 - accuracy: 0.7410 - val_loss: 0.5470 - val_accuracy: 0.7662
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5196 - accuracy: 0.7508 - val_loss: 0.5314 - val_accuracy: 0.7532
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5075 - accuracy: 0.7606 - val_loss: 0.5199 - val_accuracy: 0.7532
Epoch 8/10
20/20 [=

## what's the keras hyperparameter tuing does :
#### 1. how to select appropriate optimizer
#### 2. No. of nodes in a layer
#### 3. how to select no. of layers
#### 4. All in all one model 

In [13]:
import kerastuner as kt

C:\Users\hp\AppData\Local\Temp\ipykernel_11600\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [14]:
def build_model(hp):
    model = Sequential() 
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer = hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [15]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [16]:
tuner.search(X_train,y_train,
             epochs =5,
             validation_data=(X_test, y_test))

In [17]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0].values

In [18]:
best_hyperparameters

{'optimizer': 'rmsprop'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(X_train,y_train, batch_size=32, epochs =100,initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 16ms/step - loss: 0.5324 - accuracy: 0.7541 - val_loss: 0.5154 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5111 - accuracy: 0.7720 - val_loss: 0.4970 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4952 - accuracy: 0.7736 - val_loss: 0.4868 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4845 - accuracy: 0.7720 - val_loss: 0.4799 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4770 - accuracy: 0.7736 - val_loss: 0.4752 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4713 - accuracy: 0.7736 - val_loss: 0.4722 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4670 - accuracy: 0.7769 - val_loss: 0.4705 - val_accuracy: 0.8052
Epoch 14

In [37]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('layer', min_value=8, max_value=128, step=8)
    model.add(Dense(units=units, activation='relu', input_dim =8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [38]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                       directory='mydir2',
                       project_name='pukar')

In [39]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 08s


In [40]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0].values

In [41]:
best_hyperparameters

{'layer': 128}